In [28]:
import labelme
import json
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


# Load JSON file
json_path = "./Pictures/1.json"
with open(json_path, "r") as f:
    label_data = json.load(f)

# Load image
image_path = "./Pictures/1.jpg"

import cv2

image_path = "/mnt/data/image.png"
image = cv2.imread(image_path)
image = np.array(image)
print(image.shape)
h, w = image.shape[:2]
center = (w / 2, h / 2)

# Define rotation parameters
angle = 180

# Perform rotation using PIL
rotated_image = image.rotate(angle, expand=False)

# Update annotation points
rotated_shapes = []
for shape in label_data["shapes"]:
    points = np.array(shape["points"])  # Extract points
    # Convert to homogeneous coordinates
    points = np.hstack([points, np.ones((points.shape[0], 1))])

    # Get the rotation matrix
    rotation_center = (image.size[0] / 2, image.size[1] / 2)
    angle_rad = np.deg2rad(-angle-90)
    rotation_matrix = np.array([
        [np.cos(angle_rad), -np.sin(angle_rad), 0],
        [np.sin(angle_rad), np.cos(angle_rad), 0],
        [0, 0, 1]
    ])

    # Apply rotation
    translation_to_center = np.array([
        [1, 0, -rotation_center[0]],
        [0, 1, -rotation_center[1]],
        [0, 0, 1]
    ])
    translation_back = np.array([
        [1, 0, rotation_center[0]],
        [0, 1, rotation_center[1]],
        [0, 0, 1]
    ])
    transformation = translation_back @ rotation_matrix  @ translation_to_center
    rotated_points = (transformation @ points.T).T

    # Update shapes with new points
    shape["points"] = rotated_points[:, :2].tolist()
    rotated_shapes.append(shape)

# Update JSON with rotated shapes
label_data["shapes"] = rotated_shapes

# Save rotated image and JSON
rotated_image_path = "rotated_image.jpg"
rotated_json_path = "rotated_labels.json"

rotated_image.save(rotated_image_path)
label_data["imagePath"] = rotated_image_path
with open(rotated_json_path, "w") as f:
    json.dump(label_data, f)

# Visualization (optional)
def visualize_rotated(image, shapes):
    plt.imshow(image)
    for shape in shapes:
        points = np.array(shape["points"])
        plt.plot(*zip(*(points.tolist() + [points[0].tolist()])), c="red")
    plt.axis("off")
    plt.show()

# Show rotated image with annotations
visualize_rotated(rotated_image, rotated_shapes)

()


ValueError: not enough values to unpack (expected 2, got 0)